In [25]:
import random
import mph
import numpy as np
import matplotlib.pyplot as plt
import os    
import time
from tqdm.notebook import tqdm
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'

path = 'D:/Josh/Fall 2021/ML-LWA/baseline.mph'

scattering = [1,2,5,6,7,231,233,236,237]

In [26]:
def start_client(path):
    global model
    global pymodel
    global client
    client = mph.start(version = '5.3a')
    pymodel = client.load(path)
    model = pymodel.java
    
def save_model():
    model.save(path)
    
def generate_image(random_pattern):
    adjusted = [x - 15 for x in random_pattern]
    out = np.array([1 if x in adjusted else 0 for x in np.arange(216)]).reshape([6,36])
    flipped = np.flip(out.reshape([6,36]).T,axis=0)
    return flipped
    
def simulate_random():
    global start_count
    grids_of_interest = [random.randint(15,230) for i in range(0,random.randint(1,216))]
    this_scattering = scattering + grids_of_interest
    
    model.physics("emw").feature("sctr1").selection().set(this_scattering);
    model.physics("emw").feature("ffd1").feature("ffc1").selection().set(grids_of_interest);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()

    return grids_of_interest

In [11]:
img_list = []

save_str = 'D:/Josh/Fall 2021/ML-LWA/results/' + str(int(time.time()))

start_client(path)
start_time = time.time()
for i in tqdm(np.arange(50)):
    goi = simulate_random()
    array = generate_image(goi)
    img_list.append(array)

print(time.time() - start_time, 'seconds')

with open(save_str + '.pkl', 'wb') as file:
    pickle.dump(img_list, file)

  0%|          | 0/50 [00:00<?, ?it/s]

1867.3863446712494 seconds
